# 🔍 Atelier Data Lake - Exploration des Données

Ce notebook vous guide dans l'exploration des données de notre Data Lake.


In [1]:
# Configuration et imports
import sys
sys.path.append('..')

from src.spark_session import get_spark_session
from config.settings import BRONZE_PATH, SILVER_PATH, GOLD_PATH, SOURCES

In [2]:
# Création de la session Spark
spark = get_spark_session("DataLake_Exploration")
print("✅ Session Spark créée!")

KeyboardInterrupt: 

## 1️⃣ Exploration des Sources de Données


In [ ]:
# Lecture des clients (CSV)
clients_df = spark.read.option("header", True).option("inferSchema", True).csv(str(SOURCES["csv"] / "clients.csv"))

print(f"Nombre de clients: {clients_df.count()}")
print("\nSchéma:")
clients_df.printSchema()
print("\nAperçu:")
clients_df.show(5)


UnsupportedOperationException: getSubject is not supported

In [ ]:
# Lecture des produits (CSV)
produits_df = spark.read.option("header", True).option("inferSchema", True).csv(str(SOURCES["csv"] / "produits.csv"))

print(f"Nombre de produits: {produits_df.count()}")
produits_df.show(5)


In [ ]:
# Lecture des commandes (JSON)
from pyspark.sql.functions import explode, col

commandes_raw = spark.read.option("multiline", True).json(str(SOURCES["json"] / "commandes.json"))
commandes_df = commandes_raw.select(explode("commandes").alias("c")).select("c.*")

print(f"Nombre de commandes: {commandes_df.count()}")
commandes_df.show(5)


## 2️⃣ Statistiques Descriptives


In [ ]:
# Statistiques sur les produits
produits_df.describe(["prix", "stock"]).show()


In [ ]:
# Distribution par catégorie
produits_df.groupBy("categorie").count().orderBy("count", ascending=False).show()


In [ ]:
# Distribution des clients par ville
clients_df.groupBy("ville").count().orderBy("count", ascending=False).show()


In [ ]:
# Distribution des commandes par statut
commandes_df.groupBy("statut").count().orderBy("count", ascending=False).show()


## 3️⃣ Qualité des Données


In [ ]:
# Vérifier les valeurs nulles
from pyspark.sql.functions import count, when, isnan, isnull

def check_nulls(df, df_name):
    print(f"\n=== Valeurs nulles dans {df_name} ===")
    null_counts = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])
    null_counts.show()

check_nulls(clients_df, "Clients")
check_nulls(produits_df, "Produits")


In [ ]:
# Vérifier les doublons
print(f"Clients uniques: {clients_df.select('client_id').distinct().count()} / {clients_df.count()}")
print(f"Produits uniques: {produits_df.select('produit_id').distinct().count()} / {produits_df.count()}")
print(f"Commandes uniques: {commandes_df.select('commande_id').distinct().count()} / {commandes_df.count()}")


In [ ]:
# Fermeture de la session
spark.stop()
print("Session Spark fermée.")
